In [1]:
import numpy as np

In [2]:
with open('movie-xid.npy','rb') as f:
  xid=np.load(f,allow_pickle=True)
with open('movie-maskid.npy','rb') as f:
  maskid=np.load(f,allow_pickle=True)
with open('movie-labelsid.npy','rb') as f:
  labelid=np.load(f,allow_pickle=True)

In [3]:
xid

array([[  101,   138,  1326, ...,     0,     0,     0],
       [  101,   138,  1326, ...,     0,     0,     0],
       [  101,   138,  1326, ...,     0,     0,     0],
       ...,
       [  101,   170, 25247, ...,     0,     0,     0],
       [  101,   170, 25247, ...,     0,     0,     0],
       [  101, 22572, 12148, ...,     0,     0,     0]])

In [4]:
maskid

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [5]:
labelid

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]])

In [6]:
#now we should create a tensorflow datasetobject from tensor slices method
import tensorflow as tf

Using TensorFlow backend.


In [7]:
dataset=tf.data.Dataset.from_tensor_slices((xid,maskid,labelid
                                           ))

In [8]:
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(512,), dtype=tf.int32, name=None), TensorSpec(shape=(512,), dtype=tf.int32, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [9]:
#for tensorflow we should take the inputs and outputs
def map(inputid,mask,labels):
    return {'input_id':inputid,
           'attention_mask':mask},labels

In [10]:
dataset=dataset.map(map)

In [11]:
dataset

<MapDataset element_spec=({'input_id': TensorSpec(shape=(512,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.int32, name=None)}, TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [12]:
batchsize=16

In [13]:
#shuffling and batching the dataset
dataset=dataset.shuffle(10000).batch(batchsize,drop_remainder=True)

In [14]:
dataset.take(1) #we can see batched tensors of 16

<TakeDataset element_spec=({'input_id': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

In [15]:
#splitting the traintest dataset
split=0.9

In [16]:
size=int((xid.shape[0]/batchsize)*split)

In [17]:
trainds=dataset.take(size)
valds=dataset.skip(size)

In [18]:
#our datasets are fully created now we can save them
tf.data.experimental.save(trainds,'train')
tf.data.experimental.save(valds,'test')

In [19]:
#TO FIND THE TF ELEMENt spec
trainds.element_spec

({'input_id': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)},
 TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [20]:
valds.element_spec

({'input_id': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)},
 TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))